# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [1]:
import numpy as np

### ESOINN node class
##### Fields
* feature_vector – weights
* accamulate_signals – number of signals
* total_points – points $\neq$ number of signals
* density – mean accumulated signals
* subclass_id – mark for subclass

In [2]:
class ESOINN_Node:
    def __init__(self, feature_vector=()):
        self.feature_vector = np.array(feature_vector, dtype=float)  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1
    
    def update_accamulate_signals(self, n=1):
        self.accamulate_signals += 1
    
    def update_points(self, points):
        self.total_points += points
        
    def update_density(self, coeff=None):
        self.density = self.total_points/(coeff if coeff else self.accamulate_signals)
        
    def update_feature_vector(self, signal, coeff):
        self.feature_vector += coeff*(signal - self.feature_vector)

### ESOINN Neural Network class
To start lerning use: `fit()` method, for clasterization use `predict()`.

##### Params:
To create new `EnhancedSelfOrganizingIncrementalNN` object – initialize it with first two nodes (`init_nodes`) randomly.

##### Hiperparams:
* `C1`, `C2` – coefficents for noise deletion.
* `learning_step` – number of iterations before remove old ages and find classes ($\lambda$ in literature).
* `max_age` – for edges.
* `forget` – specify which N is used in density calculation.
* `metrics` – lambda(x, y, axis)
* `radius_cut_off` – degree of neighbors' nodes.
* `learning_rate_winner` 
* `learning_rate_winner_neighbor`

##### Fields:
* `ids` – last given id for nodes (should be unique).

In [14]:
class EnhancedSelfOrganizingIncrementalNN:
    def __init__(self, init_nodes, C1=0.001, C2=1, learning_step=200, max_age=50, 
                 metrics=lambda x,y,axis=0: np.sqrt(np.sum(np.square(np.array(x) - np.array(y)), axis=axis)),
                 forget=False, radius_cut_off=1, learning_rate_winner=lambda t: 1/t, 
                 learning_rate_winner_neighbor=lambda t: 1/(100*t)):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age = max_age
        self.count_signals = 2
        self.metrics = metrics
        self.forget = forget
        self.unique_id = 2
        self.rc = radius_cut_off
        self.rate = learning_rate_winner
        self.rate_neighbor = learning_rate_winner_neighbor
        
        self.nodes = {i: ESOINN_Node(init_nodes[i]) for i in (0, 1)}
        # @TODO: use { node_id: { neighbor_id: age } } instead of self.neighbors, self.edges
        self.neighbors = {}  # key = id, value = set of neighbors' ids
        self.edges = {}  # key = tuple(2), where t[0] < t[1], value = age/None
    
    def fit(self, input_signal):
        self.count_signals += 1
        
        winners_ids, distances = self.find_winners(input_signal)
        # @TODO: do not use thesholds list, calc it inplace
        thresholds = [self.calc_threshold(input_signal, winners_ids[i]) for i in (0, 1)]
        if distances[0] > thresholds[0] or distances[1] > thresholds[1]:
            self.create_node(input_signal)
            return
        
        self.update_edges_age(winners_ids[0])
        self.build_connection(winners_ids)
        
        # do it one time only 'cause it takes long time
        winner_neighbors = self.find_neighbors(winners_ids[0], depth=self.rc)
        # @CHECKME: обновление количества побед нейрона - не знаю куда поставить, но точно до подсчета плотности
        self.nodes[winners_ids[0]].update_accamulate_signals()
        
        self.update_neuron_density(winners_ids[0], winner_neighbors)
        self.update_neurons_feature_vector(winners_ids[0], input_signal, winner_neighbors)
        self.remove_old_ages()
        
        if not self.count_signals % self.learning_step:
            self.separate_subclasses()  # @TODO: algorithm 3.1
            self.remove_noise()  # @TODO: noize removal
    
    def find_winners(self, input_signal):
        # @FIXME: inf coef and separate variables for each winner
        winner1 = float('inf')
        winner1_id = -1
        winner2 = float('inf')
        winner2_id = -1
        for node_id in self.nodes:
            dist = self.metrics(input_signal, self.nodes[node_id].feature_vector)
            if dist <= winner1:
                winner1, winner2 = dist, winner1
                winner2_id = winner1_id
                winner1_id = node_id
            elif dist < winner2:
                winner2 = dist
                winner2_id = node_id
        return [winner1_id, winner2_id], [winner1, winner2]
    
    def find_neighbors(self, start_node_id, depth=1) -> set():
        visited = {start_node_id}
        queue = list(self.neighbors.get(start_node_id, set()) - visited)
        while depth and queue:
            depth -= 1
            for vertex in queue.copy():  # @FIXME: do not use copy!
                visited.add(vertex)
                queue.remove(vertex)
                queue.extend([node for node in self.neighbors[vertex] - visited if node not in visited])
        return visited - {start_node_id}    
    
    def calc_threshold(self, input_signal, winner_id):
        neighbors = self.neighbors.get(winner_id, None)
        if neighbors:
            return np.max([
                self.metrics(self.nodes[winner_id].feature_vector, self.nodes[neighbor_id].feature_vector) 
                for neighbor_id in self.find_neighbors(winner_id, depth=self.rc)
            ])
        else:
            return self.find_winners(self.nodes[winner_id].feature_vector)[1][1]  # 'cause first winner is always current node
    
    def create_node(self, input_signal):  
        self.nodes[self.unique_id] = ESOINN_Node(input_signal)
        self.unique_id += 1  # to provide unique ids for each neuron
    
    def update_edges_age(self, node_id, step=1):
        neighbors = self.find_neighbors(node_id, depth=self.rc)
        for neighbor_id in neighbors:
            pair_id = min(node_id, neighbor_id), max(node_id, neighbor_id)
            self.edges[pair_id] += 1
                
    # algorithm 3.2
    def build_connection(self, winners_ids):
        winners_classes = [self.nodes[winners_ids[i]].subclass_id for i in [0, 1]]
        if winners_classes[0] == -1 or winners_classes[1] == -1 or winners_classes[0] == winners_classes[1] or self.merge_subclass_condition(winners_ids):
            self.combine_subclasses(winners_ids)
            self.create_edges(winners_ids)
        else:
            self.remove_edges(winners_ids)
                
    def create_edges(self, nodes_ids):
        for node_id in nodes_ids:
            if node_id not in self.neighbors:
                self.neighbors[node_id] = set()
            for insert_id in nodes_ids:
                if insert_id != node_id:
                    self.neighbors[node_id] |= {insert_id}
                    nodes_pair = (min(node_id, insert_id), max(node_id, insert_id))
                    self.edges[nodes_pair] = 0
                    
    # @CHECKME: with index usage
#     def create_edges(self, nodes_ids):
#         for node_index in range(len(nodes_ids)):
#             if nodes_ids[node_index] not in self.neighbors:
#                 self.neighbors[nodes_ids[node_index]] = set()
#             for insert_index in range(node_index+1, len(nodes_ids)):
#                 if insert_index != node_index:
#                     self.neighbors[nodes_ids[node_index]] |= {nodes_ids[insert_index]}
#                     nodes_pair = (nodes_ids[node_index], nodes_ids[insert_index])
#                     self.edges[nodes_pair] = 0
    
    # @FIXME: keys repeats in for cycle
    def remove_edges(self, nodes_ids):
        for node_id in nodes_ids:
            for remove_id in nodes_ids:
                if remove_id != node_id and remove_id in self.neighbors[node_id]:
                    self.neighbors[node_id] -= {remove_id}
                    nodes_pair = (min(node_id, remove_id), max(node_id, remove_id))
                    if nodes_pair in self.edges:
                        del self.edges[nodes_pair]
            if not self.neighbors[node_id]:
                del self.neighbors[node_id]

    # @CHECKME: with index usage
#     def remove_edges(self, nodes_ids):
#         for node_index in range(len(nodes_ids)):
#             for remove_index in range(node_index+1, len(nodes_ids)):
#                 if remove_index != node_index and nodes_ids[remove_index] in self.neighbors[nodes_ids[node_index]]:
#                     self.neighbors[nodes_ids[node_index]] -= {nodes_ids[remove_index]}
#                     nodes_pair = (nodes_ids[node_index], nodes_ids[remove_index])
#                     if nodes_pair in self.edges:
#                         del self.edges[nodes_pair]
#             if not self.neighbors[nodes_ids[node_index]]:
#                 del self.neighbors[nodes_ids[node_index]]
                         
    def update_node_points(self, winner_id, neighbors):
        if neighbors:
            mean_dist2neighbors = 1/len(neighbors)*np.sum([
                self.metrics(self.nodes[winner_id].feature_vector, self.nodes[neighbor_id].feature_vector)
                for neighbor_id in neighbors
            ])
        self.nodes[winner_id].update_points(1/(1 + (0 if not neighbors else mean_dist2neighbors))**2)

    def update_neuron_density(self, winner_id, neighbors):
        self.update_node_points(winner_id, neighbors)
        self.nodes[winner_id].update_density((self.count_signals if self.forget else None))

    def update_neurons_feature_vector(self, winner_id, input_signal, neighbors):
        acc_signal = self.nodes[winner_id].accamulate_signals
        self.nodes[winner_id].update_feature_vector(input_signal, self.rate(acc_signal))
        coeff_neighbors = self.rate_neighbor(acc_signal)
        for neighbor_id in neighbors:
            self.nodes[neighbor_id].update_feature_vector(input_signal, coeff_neighbors)
    
    # @CHECKME: use remove_edges() - but no test done
    def remove_old_ages(self):
        for edge in self.edges.copy():
            if self.edges[edge] > self.max_age:
                self.remove_edges(edge)
    
    def calc_mean_density_in_subclass(self, node_id):
        neighbors = self.find_neighbors(node_id, depth=len(self.neighbors[node_id]))
        return 1/len(neighbors)*np.sum([self.nodes[node_id].density for node_id in neighbors])
    
    def calc_alpha(self, node_id, apex_density):
        mean_density = calc_mean_density_in_subclass(node_id)
        if 2*mean_density >= apex_density:
            return 0
        elif 3*mean_density >= apex_density:
            return 0.5
        return 1
    
    def merge_subclass_condition(self, winners_ids):
        min_winners_density = min(self.nodes[winners_ids[0]].density, self.nodes[winners_ids[1]].density)
        return (
            min_winners_density > self.calc_alpha(
                winners_ids[0],
                self.nodes[self.nodes[winners_ids[0]].subclass_id].density
            )
        ) or (
            min_winners_density > self.calc_alpha(
                winners_ids[1], 
                self.nodes[self.nodes[winners_ids[1]].subclass_id].density
            )
        )
    
    def change_class_id(self, node_id, class_id):
        neighbors = self.find_neighbors(node_id, depth=-1)
        for nieghbor_id in neighbors:
            self.nodes[neighbor_id].subclass_id = class_id
    
    def combine_subclasses(self, nodes_ids):
        nodes = [self.nodes[nodes_ids[0]], self.nodes[nodes_ids[1]]]
        subclass_ids = [nodes[0].subclass_id, nodes[1].subclass_id]
        if subclass_ids[0] == -1 and subclass_ids[1] == -1:
            for node in nodes:
                node.subclass_id = nodes_ids[0]
        elif subclass_ids[0] != subclass_ids[1]:
            if subclass_ids[0] == -1:
                change_class_id(nodes_ids[0], subclass_ids[1])
            else:
                change_class_id(nodes_ids[1], subclass_ids[0])
    
    def is_extremum(self, node_id) -> int:
        neighbors = self.find_neighbors(node_id)
        current_density = self.nodes[node_id].density
        local_min = False
        local_max = False
        for neighbor_id in neighbors:
            if local_min and local_max:
                return
            neighbor_density = self.nodes[neighbor_id].density
            if current_density > neighbor_density:
                local_max = True
            elif current_density < neighbor_density:
                local_min = True
            else:
                raise RuntimeError("Equal nodes' density!")
        if local_min and not local_max:
            return -1
        elif local_max and not local_min:
            return 1
        
    # @TODO: paste working algorithm here and adapt it for usage in class
    # @FIXME: improve search by removing multy vertex addition in queue
    def find_neighbors_local_maxes(self, node_id):
        # @CHECKME: remove this condition if it used previously in separate_subclasses()
        if self.is_extremum(node_id) == 1:
            return {node_id}
        
        apexes = set()
        visited = {node_id}
        queue = list(self.neighbors.get(node_id, set()) - visited)
        for vertex in queue:
            is_local_max = True
            visited.add(vertex)
            vertex_density = self.nodes[vertex].density
            for neighbor_id in self.neighbors[vertex]:
                if self.nodes[neighbor_id].density > vertex_density:
                    if neighbor_id not in visited:
                        queue.append[neighbor_id]
                elif is_local_max:
                    is_local_max = False
                visited.add(neighbor_id)
            if is_local_max:
                apexes.add(vertex)
        return apexes
    
    def mark_subclasses(self):
        pass  # @TODO: if local max found, mark all !min as one class
    
    # algorithm 3.1
    def separate_subclasses(self, visited=set()):
        for node_id in self.nodes():
            node_is_extremum = self.is_extremum(node_id)
            if not node_is_extremum:
                pass
            elif node_is_extremum == 1:
                self.nodes[node_id].subclass_id = node_id
                visited.add(node_id)
                neighbors = self.find_neighbors(node_id)
                for neighbor_id
            else:
                pass
    
    def remove_noise(self):
        pass # @TODO

    def predict(self, input_signal):
        pass  # @TODO: make predictions

    def update(self):
        pass  # @TODO: update topology
    
    def current_state(self):
        return {
            'count_signals': self.count_signals,
            'C1': self.C1,
            'C2': self.C2,
            'lambda': self.learning_step,
            'forget': self.forget,
            'max_age': self.max_age,
            'metrics': self.metrics,
            'learning_rate_winner': self.rate,
            'learning_rate_winner_neighbor': self.rate_neighbor,
            'nodes': self.nodes,  # think about it
            'neighbors': self.neighbors,
            'edges': self.edges
        }

### tests
@TODO: Should be automated.

In [4]:
input_signal = np.array([2, 2])  # [2,19] for node&edge creation
nn = EnhancedSelfOrganizingIncrementalNN([[1, 2], [5, 2]])

nn.fit(input_signal)
# test for old edge removal
# nn.edges[(0,1)] = 51
# nn.remove_old_ages()

nn_info = nn.current_state()  # this is more correct

print("TEST nodes feature vector and density:")
for i in nn_info['nodes'].keys():
    print(f"node {i} : {nn_info['nodes'][i].feature_vector} | density {nn_info['nodes'][i].density}")

print("\nTEST neighbours for nodes:")
for i in range(len(nn_info['neighbors'])):
    print(f"neighbors for node {i} = {nn_info['neighbors'].get(i, None)}")

print("\nTEST edges between nodes:")
print(f"edge between winners (node 0, node 1) = {nn_info['edges'].get((0,1))}")
print(f"edge between winners (None) = {nn_info['edges'].get(())}")

TEST nodes feature vector and density:
node 0 : [ 2.  2.] | density 0.04
node 1 : [ 4.97  2.  ] | density 0

TEST neighbours for nodes:
neighbors for node 0 = {1}
neighbors for node 1 = {0}

TEST edges between nodes:
edge between winners (node 0, node 1) = 0
edge between winners (None) = None


In [120]:
import pickle

with open(r"src/suspended_undirected_graph", "rb") as file:
    g = pickle.load(file)
g['density'][13] = 4
g
# with open(r"src/suspended_undirected_graph", "wb") as file:
#     pickle.dump(g, file)

{'density': {1: 15,
  2: 10,
  3: 2,
  4: 2,
  5: 5,
  6: 7,
  7: 4,
  8: 3,
  9: 3,
  10: 3,
  11: 1,
  12: 2,
  13: 4,
  14: 3,
  15: 4,
  16: 2,
  17: 5,
  18: 6,
  19: 5,
  20: 4,
  21: 3,
  22: 2,
  23: 1,
  24: 2,
  25: 3,
  26: 1,
  27: 1,
  28: 1,
  29: 2,
  30: 2,
  31: 1,
  32: 2,
  33: 1,
  34: 2},
 'nodes': {1: {16, 17, 26, 27, 28},
  2: {12, 13, 30, 31, 34},
  3: {11},
  4: {11},
  5: {11, 12, 13, 14, 15},
  6: {13, 14, 15, 18, 30},
  7: {23, 24, 25, 26, 27},
  8: {26, 27, 28, 29, 30, 31},
  9: {29, 30, 31, 32},
  10: {31, 32},
  11: {3, 4, 5},
  12: {2, 5, 13, 14, 15},
  13: {2, 5, 6, 12, 14, 16, 17, 26},
  14: {5, 6, 12, 13, 15, 16, 17},
  15: {5, 6, 12, 14, 16, 17},
  16: {1, 13, 14, 15},
  17: {1, 13, 14, 15, 18},
  18: {6, 17, 19},
  19: {18, 20, 21},
  20: {19, 21},
  21: {19, 20, 22, 23},
  22: {21, 23},
  23: {7, 21, 22, 24},
  24: {7, 23, 25},
  25: {7, 24, 26, 27},
  26: {1, 7, 8, 13, 25, 29, 30},
  27: {1, 7, 8, 25, 29, 30},
  28: {1, 8, 29, 30},
  29: {8, 9, 26

In [261]:
def find_neighbors_local_maxes(g, node_id):
    apexes = set()
    visited = {node_id}
    
    queue = []
    node_density = g['density'][node_id]
    for neighbor_id in g['nodes'].get(node_id, set()) - visited:
        if g['density'][neighbor_id] > node_density:
            queue.append(neighbor_id)
        visited.add(neighbor_id)
    
    for vertex in queue:
        is_local_max = True
        vertex_density = g['density'][vertex]
        for neighbor_id in g['nodes'][vertex]:
            if g['density'][neighbor_id] > vertex_density:
                if neighbor_id not in visited:
                    queue.append(neighbor_id)
                is_local_max = False
            visited.add(neighbor_id)
        if is_local_max:
            apexes.add(vertex)
        visited.add(vertex)
    return apexes

In [263]:
%%timeit

d = {}
for i in range(34):
    d[i+1] = find_neighbors_local_maxes(g, i+1)
#     print(f"{i+1:<6}{find_neighbors_local_maxes(g, i+1)}")
# print(d == check)

1000 loops, best of 3: 292 µs per loop


In [134]:
check = {1: set(),
 2: set(),
 3: set(),
 4: set(),
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: {3, 4, 5},
 12: {1, 2, 5, 6},
 13: {1, 2, 5, 6},
 14: {1, 2, 5, 6},
 15: {1, 5, 6},
 16: {1, 2, 5, 6},
 17: {1, 6},
 18: {6},
 19: {6},
 20: {6},
 21: {6},
 22: {6},
 23: {6, 7},
 24: {7},
 25: {7},
 26: {1, 2, 5, 6, 7, 8, 9},
 27: {1, 2, 6, 7, 8, 9},
 28: {1, 2, 6, 8, 9},
 29: {8, 9},
 30: {2, 6, 8, 9},
 31: {2, 6, 8, 9, 10},
 32: {9, 10},
 33: {2, 9, 10},
 34: {2}}